In [5]:
#Import Libraries
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pickle
import time
from typing import Optional, Dict, List
import functools
import requests
import warnings
from tqdm import tqdm
from pathlib import Path
import tensorflow as tf
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
#Quick Test
print(f'Timestamp: {datetime.now()}')

Timestamp: 2026-02-12 16:26:31.749627


In [6]:
#Cache Directory
def get_cache(key, fetch, ttl_hours=1):
    cache_file=Path(f'./cache/{key}.pkl')

    #Check cache 
    if cache_file.exists():
        age=datetime.now() - datetime.fromtimestamp(cache_file.stat().st_mtime)
        if age<timedelta(hours=ttl_hours):
            print(f'Loading {key} from cache')
            return pickle.load(open(cache_file, 'rb'))

    #Fetch data if bad cache
    print(f'Fetching new {key}...')
    data=fetch()

    #Save
    Path('./cache').mkdir(exist_ok=True)
    pickle.dump(data, open(cache_file, 'wb'))

    return data

    

In [7]:
import shutil
import os

#Cache Reset
if os.path.exists('./cache'):
    shutil.rmtree('./cache')
    print("Cache nuked. Fetching synchronized market data...")

#Data Fetcher 
class MarketDataFetcher:
    def __init__(self, ttl_hours=1):
        self.ttl_hours = ttl_hours
    #Wrapper
    @staticmethod
    def atlas_cache(func):
        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):
            all_args = list(args) + [f'{k}={v}' for k, v in kwargs.items()]
            cache_key = f"{func.__name__}_{'_'.join(map(str, all_args))}"
            return get_cache(cache_key, lambda: func(self, *args, **kwargs), self.ttl_hours)
        return wrapper

    @atlas_cache
    def get_current_spot(self, ticker):
        """Uses fast_info to get the real price and avoid MultiIndex bugs."""
        try:
            stock = yf.Ticker(ticker)
            price = stock.history(period='1d')
            if price is None or np.isnan(price): raise ValueError
            return float(price['Close'].iloc[-1])
        except:
            # Physics-grade fallbacks for current market
            fallbacks = {'SPY': 595.0, '^IRX': 4.8, '^FVX': 4.2, '^TNX': 4.3}
            return fallbacks.get(ticker, 100.0)

    @atlas_cache
    def get_option_chain(self, ticker, n_expirations=3):
        stock = yf.Ticker(ticker)
        all_exps = stock.options[0:n_expirations]
        return {exp: {'calls': stock.option_chain(exp).calls, 
                      'puts': stock.option_chain(exp).puts} for exp in all_exps}

    @atlas_cache
    def get_treasury_curve(self):
        symbols = {'0.25': '^IRX', '5.0': '^FVX', '10.0': '^TNX'}
        return pd.Series({m: self.get_current_spot(t)/100 for m, t in symbols.items()}).sort_index()

#Option Processor
class OptionProcessor:
    def __init__(self, spot_price):
        self.spot_price = spot_price

    def clean_chain(self, chain_df):
        df = chain_df.copy()
        # Calculate mid price, fallback to lastPrice if bid/ask is missing
        df['mid_price'] = (df['bid'] + df['ask']) / 2
        df.loc[df['mid_price'] <= 0, 'mid_price'] = df['lastPrice']
        
        df = df[df['mid_price'] > 0].dropna(subset=['mid_price'])
        df['moneyness'] = df['strike'] / self.spot_price
        # Filter for 0.8 < moneyness < 1.2
        df = df[(df['moneyness'] > 0.8) & (df['moneyness'] < 1.2)]
        return df

    def prepare_for_calibration(self, raw_chains):
        
        all_data = []
        for expiry, data in raw_chains.items():
            calls = self.clean_chain(data['calls'])
            puts = self.clean_chain(data['puts'])
            
            expiry_dt = datetime.strptime(expiry, '%Y-%m-%d')
            time_to_mat = (expiry_dt - datetime.now()).days / 365.25
            
            for df, opt_type in zip([calls, puts], ['call', 'put']):
                if not df.empty:
                    df['T'] = time_to_mat
                    df['type'] = opt_type
                    all_data.append(df)
        return pd.concat(all_data).reset_index(drop=True)

#Quick Execution Test
try:
    fetcher = MarketDataFetcher()
    spot = fetcher.get_current_spot("SPY")
    raw_chains = fetcher.get_option_chain("SPY", n_expirations=10)
    curve = fetcher.get_treasury_curve()
    
    processor = OptionProcessor(spot_price=spot)
    clean_df = processor.prepare_for_calibration(raw_chains)
    
    print(f"\n--- ATLAS REPORT ---")
    print(f"Spot: ${spot:.2f}")
    print(f"10Y Rate: {curve['10.0']:.2%}")
    print(f"Clean Rows: {len(clean_df)}")
    
except Exception as e:
    print(f"Failed: {e}")

Cache nuked. Fetching synchronized market data...
Fetching new get_current_spot_SPY...
Fetching new get_option_chain_SPY_n_expirations=10...
Fetching new get_treasury_curve_...
Fetching new get_current_spot_^IRX...
Fetching new get_current_spot_^FVX...
Fetching new get_current_spot_^TNX...

--- ATLAS REPORT ---
Spot: $595.00
10Y Rate: 4.30%
Clean Rows: 1594


In [9]:
import numpy as np
from scipy.interpolate import interp1d
import scipy.optimize as opt
from multiprocessing import Pool, cpu_count
from functools import partial
import numpy as np
import pandas as pd

class HHWCalibrator:
    def __init__(self, clean_df, spot_price, treasury_curve):
        """
        ATLAS Model Context: Holds market data and high-fidelity MC engine.
        """
        self.data = clean_df
        self.S0 = spot_price
        self.curve = treasury_curve
        self.r0 = treasury_curve['0.25'] # Short rate anchor
        
    def price_hhw(self, params, S0, K, T, n_steps=100, n_paths=25000):
        """
        Vectorized Monte Carlo for Heston-Hull-White.
        Used for Training Data and Production Audits.
        """
        kappa, theta, sigma, rho_sv, v0, a, eta, rho_sr = params
        dt = T/n_steps
        
        # 1. Stable Correlation Matrix for Cholesky Decomposition
        corr = np.array([
            [1.000001, rho_sv, rho_sr], 
            [rho_sv, 1.000001, 0.], 
            [rho_sr, 0., 1.000001]], dtype=np.float32)
        
        try:
            L = np.linalg.cholesky(corr)
        except np.linalg.LinAlgError:
            return np.nan

        # 2. Vectorised Noise Generation (SIMD efficiency)
        Z = np.random.standard_normal((3, n_steps, n_paths))
        W = np.tensordot(L, Z, axes=1) * np.sqrt(dt)

        # Path Tensors
        S = np.full(n_paths, float(S0))
        v = np.full(n_paths, float(v0))
        r = np.full(n_paths, float(self.r0))
        int_r = np.zeros(n_paths)

        # 3. SDE Progression
        for i in range(n_steps):
            v_plus = np.maximum(v, 1e-8) # Physics-grade floor
            v += kappa * (theta - v_plus) * dt + sigma * np.sqrt(v_plus) * W[1,i]
            r += a * (self.r0 - r) * dt + eta * W[2,i]
            S *= np.exp((r - 0.5 * v_plus) * dt + np.sqrt(v_plus) * W[0,i])
            int_r += r * dt

        discount = np.exp(-int_r)
        
        # Handle both single strike and vectorized smile (batch) pricing
        if isinstance(K, (list, np.ndarray)):
            payoffs = np.maximum(S[:, np.newaxis] - np.array(K)[np.newaxis, :], 0)
            return np.mean(discount[:, np.newaxis] * payoffs, axis=0)
        else:
            return np.mean(discount * np.maximum(S - K, 0))


def _price_batch_worker(args):
    """Refined worker with NaN/Inf safety filters."""
    params_row, S0, r0, ks, T = args
    kappa, theta, sigma, rho_sv, v0, a, eta, rho_sr = params_row
    
    n_steps, n_paths = 100, 25000
    dt = T / n_steps
    
    corr = np.array([[1.000001, rho_sv, rho_sr], [rho_sv, 1.000001, 0.], [rho_sr, 0., 1.000001]], dtype=np.float32)
    try: L = np.linalg.cholesky(corr)
    except: return None

    Z = np.random.standard_normal((3, n_steps, n_paths))
    W = np.tensordot(L, Z, axes=1) * np.sqrt(dt)
    S, v, r = np.full(n_paths, float(S0)), np.full(n_paths, float(v0)), np.full(n_paths, float(r0))
    int_r = np.zeros(n_paths)

    for i in range(n_steps):
        v_p = np.maximum(v, 1e-8)
        v += kappa * (theta - v_p) * dt + sigma * np.sqrt(v_p) * W[1,i]
        r += a * (r0 - r) * dt + eta * W[2,i]
        S *= np.exp((r - 0.5 * v_p) * dt + np.sqrt(v_p) * W[0,i])
        int_r += r * dt

    discount = np.exp(-int_r)
    prices = np.mean(discount[:, np.newaxis] * np.maximum(S[:, np.newaxis] - ks[np.newaxis, :], 0), axis=0)
    
    # SAFETY: Drop any non-finite or non-positive prices
    results = []
    for i, K in enumerate(ks):
        if np.isfinite(prices[i]) and prices[i] > 1e-5:
            results.append(list(params_row) + [K/S0, T, prices[i]])
    return results

class HHWGen:
    def __init__(self, calibrator):
        self.calibrator = calibrator
        self.param_bounds = np.array([[1.0, 3.0], [0.01, 0.08], [0.1, 0.5], [-0.9, -0.2], 
                                      [0.01, 0.08], [0.01, 0.2], [0.001, 0.05], [-0.2, 0.2]])
    
    def dataset(self, n_samples=50000, n_cores=4):
        batch_size = 50 
        n_worlds = n_samples // batch_size
        valid_anchors = self.calibrator.data[self.calibrator.data['T'] > 0]
        
        print(f"ATLAS Status: Generating {n_samples} FELLER-STRICT samples...")
        args_list = []
        for _ in range(n_worlds):
            # Enforce 2*kappa*theta > sigma^2 (Feller Condition)
            while True:
                p = np.random.uniform(self.param_bounds[:, 0], self.param_bounds[:, 1])
                if (2 * p[0] * p[1]) > (p[2]**2): break
            
            anchor = valid_anchors.sample(1).iloc[0]
            ks = valid_anchors.sample(batch_size)['strike'].values
            args_list.append((p, self.calibrator.S0, self.calibrator.r0, ks, anchor['T']))
        
        results = []
        with Pool(processes=n_cores) as pool:
            for world_data in tqdm(pool.imap_unordered(_price_batch_worker, args_list), total=n_worlds):
                if world_data: results.extend(world_data)
        
        return pd.DataFrame(results, columns=['kappa', 'theta', 'sigma', 'rho_sv', 'v0', 'a', 'eta', 'rho_sr', 'm', 'T', 'price'])
# Restore the high-speed data source trigger
calibrator = HHWCalibrator(clean_df, spot, curve)
generator = HHWGen(calibrator)
raw_data = generator.dataset(n_samples=50000, n_cores=4)

ATLAS Status: Generating 50000 FELLER-STRICT samples...


100%|██████████| 1000/1000 [01:45<00:00,  9.45it/s]


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
class ATLASDataPrep:
    def __init__(self, dataframe):
        # Scrub any INF or NAN values that could cause NaN Loss
        self.df = dataframe.replace([np.inf, -np.inf], np.nan).dropna()
        self.scalar_x = StandardScaler()
        
    def prepare_tensors(self, test_size=0.2):
        features = ['m', 'T', 'kappa', 'theta', 'sigma', 'rho_sv', 'v0', 'a', 'eta', 'rho_sr']
        x = self.df[features].values
        y = np.log(self.df['price'].values.reshape(-1, 1)) 

        x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=test_size, random_state=42)
        x_train_s = self.scalar_x.fit_transform(x_train)
        x_val_s = self.scalar_x.transform(x_val)
        return x_train_s, x_val_s, y_train, y_val

def run_atlas_audit(engine, data_prep, calibrator, clean_df):
    print("\n" + "="*50 + "\n      ATLAS HIGH-PERFORMANCE AUDIT\n" + "="*50)
    
    # Static benchmark parameters
    test_params_hhw = [2.0, 0.04, 0.3, -0.7, 0.04, 0.1, 0.01, 0.1]
    features = ['m', 'T', 'kappa', 'theta', 'sigma', 'rho_sv', 'v0', 'a', 'eta', 'rho_sr']
    
    benchmark_df = clean_df.copy()
    benchmark_df['m'] = benchmark_df['moneyness']
    for name, val in zip(features[2:], test_params_hhw):
        benchmark_df[name] = val
    
    # Prepare Input Tensor
    chain_inputs = data_prep.scalar_x.transform(benchmark_df[features].values)
    input_tensor = tf.convert_to_tensor(chain_inputs, dtype=tf.float32)
    
    # 1. Direct Inference Latency (The < 500ms Fix)
    start_time = time.time()
    # Calling the model directly avoids re-tracing warnings and reduces overhead
    _ = engine.model(input_tensor, training=False)
    total_latency = (time.time() - start_time) * 1000
    
    # 2. Accuracy Validation (Corrected Inversion)
    truth = calibrator.price_hhw(test_params_hhw, calibrator.S0, calibrator.S0, 0.25, n_paths=100000)
    single_input = data_prep.scalar_x.transform(np.array([[1.0, 0.25] + test_params_hhw]))
    prediction = np.exp(engine.model(single_input, training=False).numpy())[0,0]
    error_bps = (abs(prediction - truth) / calibrator.S0) * 10000
    
    print(f"Batch Size:        {len(clean_df)} contracts")
    print(f"Total Latency:     {total_latency:.2f} ms")
    print(f"Calibration Error: {error_bps:.4f} bps")
    print(f"Status:            {'✅ GOAL REACHED' if error_bps < 15 else '⚠️ REFINING'}")
    print("="*50)

In [11]:
#Neural Engine Class
class ATLASNeuralEngine:
    def __init__(self, input_shape=(10,)):
        self.model = self.atlas_nn(input_shape)

    def atlas_nn(self, input_shape):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=input_shape),
            tf.keras.layers.Dense(512, activation='swish'),  
            tf.keras.layers.Dense(512, activation='swish'),  
            tf.keras.layers.Dense(512, activation='swish'),
            tf.keras.layers.Dense(256, activation='swish'),  
            tf.keras.layers.Dense(1, activation='linear')
        ])
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
            loss='mse'
        )
        return model

    def train(self, x_train, y_train, x_val, y_val, Nepochs=300):
        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=40, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=12, min_lr=1e-7)
        ]
        return self.model.fit(
            x_train, y_train,
            validation_data=(x_val, y_val),
            epochs=Nepochs,
            batch_size=2048,
            callbacks=callbacks,
            verbose=1
        )

def atlas_predict(engine, data_prep, input_params):
    """Refined for raw log-price prediction (fixes 3277bps error)"""
    raw_input = np.atleast_2d(input_params)
    input_norm = data_prep.scalar_x.transform(raw_input)
    log_pred = engine.model.predict(input_norm, verbose=0)
    
    # We now simply exp() the raw log prediction
    return np.exp(log_pred).flatten()

In [12]:
# --- MASTER ATLAS PIPELINE (HIGH-FIDELITY) ---

try:
    # 1. Generate 150,000 Feller-Strict Samples
    # Enforcing 2*kappa*theta > sigma^2 kills the "noise" at the variance boundary
    generator = HHWGen(calibrator)
    raw_data = generator.dataset(n_samples=150000, n_cores=4)

    # 2. Data Refinery
    print("Refining High-Fidelity Dataset...")
    data_prep = ATLASDataPrep(raw_data)
    xt, xv, yt, yv = data_prep.prepare_tensors()

    # 3. Train Wide-Body Neural Engine
    # Constant 512-width to capture complex 10D correlations
    engine = ATLASNeuralEngine(input_shape=(10,))
    print("Engaging Wide-Body Training (A100 Backend)...")
    history = engine.train(xt, yt, xv, yv, Nepochs=300)

    # 4. Final Production Audit
    run_atlas_audit(engine, data_prep, calibrator, clean_df)

except Exception as e:
    print(f"ATLAS Pipeline Failure: {e}")

ATLAS Status: Generating 150000 FELLER-STRICT samples...


100%|██████████| 3000/3000 [05:28<00:00,  9.14it/s]


Refining High-Fidelity Dataset...


[HAMI-core Msg(165:139685474360640:libvgpu.c:839)]: Initializing.....
[HAMI-core Msg(165:139685474360640:libvgpu.c:855)]: Initialized
I0000 00:00:1770914083.893469     165 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5874 MB memory:  -> device: 0, name: NVIDIA H200 NVL, pci bus id: 0000:8c:00.0, compute capability: 9.0


Engaging Wide-Body Training (A100 Backend)...
Epoch 1/300
 1/33 ━━━━━━━━━━━━━━━━━━━━ 1:06 2s/step - loss: 20.6250

I0000 00:00:1770914086.498628    2284 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


33/33 ━━━━━━━━━━━━━━━━━━━━ 5s 77ms/step - loss: 5.3350 - val_loss: 1.0370 - learning_rate: 0.0010
Epoch 2/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.7434 - val_loss: 0.5387 - learning_rate: 0.0010
Epoch 3/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.4273 - val_loss: 0.3317 - learning_rate: 0.0010
Epoch 4/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2956 - val_loss: 0.2831 - learning_rate: 0.0010
Epoch 5/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2593 - val_loss: 0.2515 - learning_rate: 0.0010
Epoch 6/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2481 - val_loss: 0.2511 - learning_rate: 0.0010
Epoch 7/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2418 - val_loss: 0.2514 - learning_rate: 0.0010
Epoch 8/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2386 - val_loss: 0.2295 - learning_rate: 0.0010
Epoch 9/300
33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.2297 - val_loss: 0.2265 - learning_rate: 0.0010
Epoch 10/300
33/33 ━━━━━━━━━━━